In [12]:
import numpy as np
data = """

"""
with open("input_23.txt", 'r') as f:
    data = f.read()

data = [x.split(" ") for x in data.split("\n") if not x == ""]

#data= data.replace("\n", "")

data

[['set', 'b', '57'],
 ['set', 'c', 'b'],
 ['jnz', 'a', '2'],
 ['jnz', '1', '5'],
 ['mul', 'b', '100'],
 ['sub', 'b', '-100000'],
 ['set', 'c', 'b'],
 ['sub', 'c', '-17000'],
 ['set', 'f', '1'],
 ['set', 'd', '2'],
 ['set', 'e', '2'],
 ['set', 'g', 'd'],
 ['mul', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '2'],
 ['set', 'f', '0'],
 ['sub', 'e', '-1'],
 ['set', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-8'],
 ['sub', 'd', '-1'],
 ['set', 'g', 'd'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-13'],
 ['jnz', 'f', '2'],
 ['sub', 'h', '-1'],
 ['set', 'g', 'b'],
 ['sub', 'g', 'c'],
 ['jnz', 'g', '2'],
 ['jnz', '1', '3'],
 ['sub', 'b', '-17'],
 ['jnz', '1', '-23']]

In [13]:
import sys
import string
def machine(pid=None):
    numSend = 0
    numMul = 0
    #print(f"hello {pid}")
    #sys.stdout.flush()
    if pid:
        registers = {"p": pid}
    else:
        registers = {}
        for x in string.ascii_lowercase[:8]:
            registers[x] = 0
    print(registers)
    ptr = 0
    while ptr < len(data):    
        op, X, *Y = data[ptr]
        Y = Y or None
        if Y:
            Y = Y[0]
            if Y.isalpha():
                Y = registers[Y] #getting register value
            Y = int(Y)
        for x in [x for x in [X, Y] if x]:
            if str(x).isalpha() and not x in registers.keys():
                registers[x] = 0

        if op == "snd":
            #send to other program
            if X in registers.keys():
                X = registers[X]
            cv.acquire()
            #print(f"sending {X}")
            box = sharedBox[(pid+1)%2]
            box.append(X)
            sharedBox[(pid+1)%2] = box
            if pid == 1:
                numSend += 1
            cv.notify()
            cv.release()
        elif op == "set":
            #set value X = val(Y)
            registers[X] = Y  
        elif op == "add":
            #add value X:= val(X) + val(Y)
            registers[X] = registers[X] + Y 
        elif op == "mul":
            #multiply value X:= val(X) * val(Y)
            registers[X] = registers[X] * Y
            numMul += 1
        elif op == "mod":
            #modulo value X:= val(X) % val(Y)
            registers[X] = registers[X] % Y
        elif op == "rcv":
            #receive from box
            cv.acquire()
            box = sharedBox[pid]
            if len(box) == 0:
                if pid == 1:
                    print(numSend)
                cv.wait()
                box = sharedBox[pid]
            registers[X] = box.pop(0)
            #print(f"received {registers[X]}")
            sharedBox[pid] = box
            cv.release()
        elif op == "jgz":
            #jump
            if X in registers.keys():
                X = registers[X]
            if int(X) > 0:
                ptr += Y
                continue
        elif op == "jnz":
            if X in registers.keys():
                X = registers[X]
            if not int(X) == 0:
                ptr += Y
                continue
        elif op == "sub":
            registers[X] = registers[X] - Y 
        else:
            print(f"unknown op {op}")
        ptr += 1
    print(numMul)

In [14]:
machine()

{'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
3025
